In [7]:
# pip install datasets transformers

from transformers import BertTokenizer

In [27]:
#from datasets import load_dataset
import pandas as pd
# Charger le dataset
dataset = pd.read_csv("C:/Users/flavi/Downloads/fifa_world_cup_2022_tweets.csv")

# print dataset info to see how the dataset is structured
print(dataset[['Sentiment']])

      Sentiment
0       neutral
1      positive
2      positive
3      positive
4      negative
...         ...
22519  positive
22520   neutral
22521  positive
22522  positive
22523   neutral

[22524 rows x 1 columns]


In [4]:
# separate tweets and sentiments
tweets = dataset['Tweet']
labels = dataset['Sentiment']
print(tweets)

0        What are we drinking today @TucanTribe \n@MadB...
1        Amazing @CanadaSoccerEN  #WorldCup2022 launch ...
2        Worth reading while watching #WorldCup2022 htt...
3        Golden Maknae shinning bright\n\nhttps://t.co/...
4        If the BBC cares so much about human rights, h...
                               ...                        
22519              Here We go World cup 2022 #WorldCup2022
22520    Anderlecht confirms former Viborg FF's Jesper ...
22521    Great thread to read before the start of #Worl...
22522    Raphinha wants Brazil to be united at the #Wor...
22523    How to buy $SOT on PinkSale?🤔\n\nHave you been...
Name: Tweet, Length: 22524, dtype: object


In [17]:
# Tokenisation
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Function to preprocess tweets
def preprocess_tweet(tweet):
    tweet = tweet.replace('\n', ' ')  # Replace newline characters with spaces
    tweet = tweet.replace('\r', ' ')  # Replace carriage returns with spaces
    tweet = tweet.replace('http', '')  # Remove URLs starting with 'http'
    tweet = tweet.replace('www', '')  # Remove URLs starting with 'www'
    tweet = tweet.replace('https', '')  # Remove URLs starting with 'https'
    tweet = tweet.replace('@', '')  # Remove '@' symbols
    tweet = tweet.replace('#', '')  # Remove '#' symbols
    tweet = ''.join([char for char in tweet if char.isalnum() or char.isspace()])  # Remove special characters
    tweet = tweet.lower()  # Convert to lowercase
    tweet = ' '.join([word for word in tweet.split() if word not in stop_words])  # Remove stopwords
    return tweet

tweets_df = dataset['Tweet'].apply(preprocess_tweet)
    

print(tweets_df)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flavi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\flavi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0        drinking today tucantribe madbears lkincalgo a...
1        amazing canadasocceren worldcup2022 launch vid...
2        worth reading watching worldcup2022 stco1sqrna...
3        golden maknae shinning bright stco4ayzbzgtx4 j...
4        bbc cares much human rights homosexual rights ...
                               ...                        
22519                       go world cup 2022 worldcup2022
22520    anderlecht confirms former viborg ffs jesper f...
22521    great thread read start worldcup2022 stcovp62j...
22522    raphinha wants brazil united worldcup2022 stco...
22523    buy sot pinksale confused buy tokens pinksale ...
Name: Tweet, Length: 22524, dtype: object


In [35]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenization
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(tweets_df)
X = tokenizer.texts_to_sequences(tweets_df)

# Padding sequences
X = pad_sequences(X, maxlen=100)

# Convert sentiment labels to numerical values
dataset['Sentiment'] = dataset['Sentiment'].map({'negative': 0, 'neutral': 1, 'positive': 2})
y = dataset['Sentiment'].values

In [43]:
## Building the LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np

# Ensure the data is a numpy array
X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_length=100))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test), verbose=1)


Epoch 1/5


AttributeError: module 'keras.src.backend' has no attribute 'convert_to_numpy'

In [ ]:
# EVALUATION

In [18]:
# 3. Model Building : simple regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=['negative', 'neutral', 'positive']))


Accuracy: 0.7094339622641509
              precision    recall  f1-score   support

    negative       0.76      0.67      0.71      1149
     neutral       0.64      0.70      0.67      1648
    positive       0.75      0.74      0.75      1708

    accuracy                           0.71      4505
   macro avg       0.72      0.71      0.71      4505
weighted avg       0.71      0.71      0.71      4505



In [2]:
## Cleaning and Tokenization:
# Remove special characters, URLs, mentions, and hashtags.
# Convert text to lowercase.
# Tokenize the tweets into words
def cleaning_prep_tweets(tweet):
    # Remove URLs
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    # Remove user @ references and '#' from hashtags
    tweet = re.sub(r'\@\w+|\#','', tweet)
    # Remove special characters, numbers, punctuations
    tweet = re.sub(r'\W', ' ', tweet)
    tweet = re.sub(r'\s+[a-zA-Z]\s+', ' ', tweet)
    tweet = re.sub(r'\^[a-zA-Z]\s+', ' ', tweet)
    tweet = re.sub(r'\s+', ' ', tweet, flags=re.I)
    tweet = re.sub(r'^b\s+', '', tweet)
    tweet = tweet.lower()
    return tweet

tweets = [preprocess_tweet(tweet) for tweet in tweets]

NameError: name 'tweets' is not defined

In [44]:
# EVALUATION DU MODELE

loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Classification report
from sklearn.metrics import classification_report
y_pred = np.argmax(model.predict(X_test), axis=-1)
print(classification_report(y_test, y_pred, target_names=['negative', 'neutral', 'positive']))


AttributeError: module 'keras.src.backend' has no attribute 'convert_to_numpy'